# Simple examples

In [1]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
text = "What would be a good company name a company that makes colorful socks?"
print(llm(text))



Colorful Kicks Co.


In [2]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=llm, prompt=prompt)
chain.run("colorful socks")

'\n\nVivid Socks.'

# Example Chain

In [5]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# This is an LLMChain to write a synopsis given a title of a play.
llm = OpenAI(temperature=.7)
template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.

Title: {title}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

# This is an LLMChain to write a review of a play given a synopsis.
llm = OpenAI(temperature=.7)
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)
review = overall_chain.run("Tragedy at sunset on the beach")



> Entering new SimpleSequentialChain chain...
 

Set on a beach at sunset, Tragedy at Sunset on the Beach follows the story of two lovers, John and Jane, who meet and fall in love in the idyllic setting of the beach. After months of blissful happiness, a sudden and unexpected tragedy strikes, overturning their lives and shaking their love. As they grapple with the aftermath of their loss, the two must find the strength within themselves to rebuild their lives and love. In their journey, they must confront the harsh reality that nothing in life is permanent and that tragedy can happen at any time. Through this story of grief, resilience, and ultimately hope, Tragedy at Sunset on the Beach is an exploration of the human spirit and its capacity to endure.


Tragedy at Sunset on the Beach is an engaging and thought-provoking play that captures the complexity of the human experience. At its core, the story follows the journey of two lovers, John and Jane, as they grapple with grief after 

# Example Agent that uses Tools

In [6]:
# Import things that are needed generically
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI

# Load the tool configs that are needed.
from langchain import LLMMathChain, SerpAPIWrapper
llm = OpenAI(temperature=0)
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    )
]

# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

agent.run("What is the age of Olivia Wilde's boyfriend raised to the 0.23 power?")



> Entering new ZeroShotAgent chain...
What is the age of Olivia Wilde's boyfriend raised to the 0.23 power?
Thought: I need to calculate this
Action: Calculator
Action Input: (age of Olivia Wilde's boyfriend)^0.23

> Entering new LLMMathChain chain...
(age of Olivia Wilde's boyfriend)^0.23

Answer: 2.8379908717450045
> Finished LLMMathChain chain.

Observation: Answer: 2.8379908717450045
Thought: I now know the final answer
Final Answer: 2.8379908717450045
> Finished ZeroShotAgent chain.


'2.8379908717450045'